# SNOWFLAKE AND AZURE OPENAI

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

import os

from snowflake.snowpark.context import get_active_session
session = get_active_session()
from openai import AzureOpenAI

os.environ["AZURE_OPENAI_ENDPOINT"] = '<>'

os.environ["AZURE_OPENAI_API_KEY"] = '<>'
deployment_name='<>'
api_version = '2023-03-15-preview'

In [ ]:
CREATE OR REPLACE NETWORK RULE CHATGPT_NETWORK_RULE
    MODE = EGRESS
    TYPE = HOST_PORT
    VALUE_LIST = ('<>.openai.azure.com'); 

In [ ]:
CREATE OR REPLACE SECRET CHATGPT_API_KEY
    TYPE = GENERIC_STRING
    SECRET_STRING='<>';

In [ ]:
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION OPENAI_INTEGRATION
    ALLOWED_NETWORK_RULES = (CHATGPT_NETWORK_RULE)
    ALLOWED_AUTHENTICATION_SECRETS = (CHATGPT_API_KEY)
    ENABLED=TRUE;


In [ ]:
CREATE OR REPLACE FUNCTION CHATGPT_IMAGE(instructions STRING, list STRING, user_context STRING)
returns string
language python
runtime_version=3.8
handler = 'ask_chatGPT'
external_access_integrations=(OPENAI_INTEGRATION)
packages = ('openai')
SECRETS = ('cred' = chatgpt_api_key )
as
$$
import _snowflake
import json
from openai import AzureOpenAI
client = AzureOpenAI(
    api_key=_snowflake.get_generic_secret_string("cred"),
    api_version='2023-03-15-preview',
    # Update Resource and Model to the base_url below
    base_url="https://hackgpt4.openai.azure.com/openai/deployments/hackdeploy/chat/completions?api-version=2024-02-15-preview"
    )
def ask_chatGPT(instructions, list_, user_context):
    response = client.chat.completions.create(
    model='{gpt-4}',
    messages = [
        {
            "role": "system",
            "content": json.dumps({
                "SYSTEM": f"Follow these: {instructions}",
                "CONTEXT_LIST": f"Use this list to select from {list_}",
                "USER_CONTEXT": f"Use this image for your response: {user_context}"
            })
        }
    ],
    max_tokens=2000 )
    return response.choices[0].message.content
$$;

In [ ]:
def list_files_in_stage(stage_name):
    query = f"LIST @{stage_name}"
    return session.sql(query).collect()

def get_presigned_url(stage_name, file_name):
    
    query = f"SELECT GET_PRESIGNED_URL(@{stage_name}, '{file_name}') AS presigned_url"
    result = session.sql(query).collect()
    return result[0]['PRESIGNED_URL'] 
    
def process_files(stage_name):
    # List files in the stage
    files = list_files_in_stage(stage_name)

    # Initialize an empty list to store file names and URLs
    data = []

    for file in files:
        file_name = file[0]  # First element is the file path
        print(f"Processing file: {file_name}")
        
        # Get the presigned URL for the file
        presigned_url = get_presigned_url(stage_name, file_name)
        
        # Add a different descriptive text based on the file name
        if "saladwithnuts" in file_name:
            description = "This food contains nuts whereas I ordered nut free option."
        elif "buritoleaking" in file_name:
            description = "This food is not made properly as expected. The food was leaking."
        file_name=file_name.split('/')[-1] 

        
        data.append({
            'file_name': file_name, 
            'presigned_url': presigned_url, 
            'description': description
        })

    # Convert the list into a pandas DataFrame
    df = pd.DataFrame(data)
    sp_df=session.create_dataframe(df)
    sp_df.write.mode("overwrite").save_as_table("Customer_Unstructured_complaints")
                                                
    return sp_df
    

In [ ]:
process_files("image_stage")

In [ ]:
image=session.file.get_stream("@IMAGE_STAGE/saladwithnuts.png", decompress=False).read() 
st.image(image)

In [ ]:
selected_image='saladwithnuts.png'
image_string = session.sql(f"""select GET_PRESIGNED_URL(@image_stage, 'saladwithnuts.png')""").collect()[0][0]
image_string

In [ ]:
cust_complaintsdf=session.table("Customer_Unstructured_complaints").to_pandas()
df_desc = cust_complaintsdf['description']
df_url=cust_complaintsdf['presigned_url']
df_filename=cust_complaintsdf['file_name']

selected_desc_index = cust_complaintsdf[cust_complaintsdf['file_name'] == selected_image].index[0]
description = df_desc.iloc[selected_desc_index]


In [ ]:

default_prompt = f"You are a customer support agent. Look at the image and recommend a resolution based on the issue reported by the user in the '{description}'. If the concern by the user does not match with the photo respond back with a message asking for further clarification."

system = st.text_area("System instructions", value=default_prompt).replace("'","")

result = session.sql(f"""SELECT chatgpt_image('{system}','{df_url}','{image_string}')""").collect()
st.header('Answer')
st.write(result[0][0].replace('"','')) 